In [1]:
import json
import numpy as np 
import pandas as pd
from loguru import logger
from modules import _get_driver_webscraping, _get_headers_webscraping, _number_of_pages, _get_html_with_many_cards, _read_results_parquet, _apply_functions_in_parallel_to_webscraping, _read_json_file, _extract_data_in_html_for_page_with_many_cards, _get_html_especifc_for_wich_property, _transform_data_to_dataframe

In [2]:
base_url = 'https://www.zapimoveis.com.br'
transaction = "venda"
type = "imoveis"
local = "rj+rio-de-janeiro"

In [3]:
#number_of_pages = _number_of_pages(base_url, transaction, type, local)
#print(number_of_pages)

number_of_pages = {'Parametros': {'Transacao': 'venda', 'Tipo': 'imoveis', 'Local': 'rj+rio-de-janeiro'}, 'Requisicao': {'Status': True}, 'Imoveis': 256990, 'Paginas': 2569}
print(json.dumps(number_of_pages, indent=4))

{
    "Parametros": {
        "Transacao": "venda",
        "Tipo": "imoveis",
        "Local": "rj+rio-de-janeiro"
    },
    "Requisicao": {
        "Status": true
    },
    "Imoveis": 256990,
    "Paginas": 2569
}


In [2]:
#list_use_in_function = list(range(1, 3))
#function_to_apply = _get_html_with_many_cards
#name_file = 'urls_imoveis'
#workers = 3
#args = [base_url, transaction, type, local]

#info_pages, erro_pages = _apply_functions_in_parallel_to_webscraping(list_use_in_function=list_use_in_function, function_to_apply=function_to_apply, name_file=name_file, key_name="url", workers=workers, args=args)
#info_pages, erro_pages

In [4]:
# with zipfile.ZipFile('data.zip', 'r') as zip_ref:
#     zip_ref.extractall()

dados_lista = _read_json_file('data/url_imoveis.json')
logger.info(f"Dados Listas Processados: {len(dados_lista)}")

2024-11-12 11:18:31.726 | SUCCESS  | modules:_read_json_file:213 - Dados Listas Processados
2024-11-12 11:18:31.728 | INFO     | __main__:<module>:5 - Dados Listas Processados: 231


In [5]:
dataframe_urls = _extract_data_in_html_for_page_with_many_cards(dados_lista)
dataframe_urls = dataframe_urls.dropna(subset=['url_imovel'])
print(dataframe_urls.shape)
dataframe_urls.head()

(8578, 9)


,pagina,anuncio_pagina,id_imovel,url_imovel,destaque,bairro,endereco,descricao,data_completa
0,0,0,2734881772,https://www.zapimoveis.com.br/imovel/venda-apa...,Super Destaque,"Copacabana, Rio de Janeiro",Rua Dias da Rocha,Apartamento à venda original com 3 quartos com...,2024-11-12
1,0,1,2460272791,https://www.zapimoveis.com.br/lancamento/venda...,Pronto para morar,"Botafogo, Rio de Janeiro",Rua Ministro Raul Fernandes,A TRANQUILIDADE NO CORAÇÃO DE BOTAFOGO!\nAPENA...,2024-11-12
2,0,2,2753248608,https://www.zapimoveis.com.br/imovel/venda-apa...,Super Destaque,"Recreio dos Bandeirantes, Rio de Janeiro",Rua Luiz Carlos Sarolli,Apartamento à Venda - 2 Quartos no Recreio dos...,2024-11-12
3,0,3,2750528798,https://www.zapimoveis.com.br/imovel/venda-apa...,Super Destaque,"Leblon, Rio de Janeiro",Rua Alberto de Faria,Apartamento á venda em Leblon 307m2-São 4 quar...,2024-11-12
4,0,4,2573819798,https://www.zapimoveis.com.br/imovel/venda-apa...,NaN,"Quintino Bocaiúva, Rio de Janeiro",Rua da República,-Cod:PPAP20178 -Rev.08Nov,2024-11-12


In [6]:
urls_de_imoveis = dataframe_urls['url_imovel'].to_list()
logger.success(f"URLs Processadas: {len(urls_de_imoveis)}")

2024-11-12 11:19:27.915 | SUCCESS  | __main__:<module>:2 - URLs Processadas: 8578


In [9]:
# list_use_in_function = urls_de_imoveis
# function_to_apply = _get_html_especifc_for_wich_property
# name_file = 'caracteristicas_imoveis'
# workers = 3
# args = []

# info_pages, erro_pages = _apply_functions_in_parallel_to_webscraping(list_use_in_function=list_use_in_function, function_to_apply=function_to_apply, name_file=name_file, key_name="url", workers=workers, args=args)
# info_pages, erro_pages

[]


Processando funções:   0%|          | 1/8578 [00:17<40:50:21, 17.14s/it]

Erro na página https://www.zapimoveis.com.br/imovel/venda-apartamento-3-quartos-com-varanda-copacabana-zona-sul-rio-de-janeiro-rj-100m2-id-2734881772/: File is not a zip file


In [2]:
lista_informacoes_imoveis, erros_imoveis = _read_results_parquet(key_name="url_imovel", name_file="caracteristicas_imoveis")
logger.success(f"Informações dos Imóveis Processadas: {len(lista_informacoes_imoveis)}")

2024-11-12 17:17:45.480 | SUCCESS  | __main__:<module>:2 - Informações dos Imóveis Processadas: 8578


In [4]:
dataframe_imoveis_tratado = _transform_data_to_dataframe(lista_informacoes_imoveis)

print("Dimensões antes da limpeza: ", dataframe_imoveis_tratado.shape)
dataframe_imoveis_tratado = dataframe_imoveis_tratado.dropna()
print("Dimensões após a limpeza: ", dataframe_imoveis_tratado.shape)
dataframe_imoveis_tratado.head()

Dimensões antes da limpeza:  (8578, 10)
Dimensões após a limpeza:  (3110, 10)


,url_imovel,condominio,endereco,area,iptu,banheiros,vagas_de_carro,quartos,suites,preco
0,https://www.zapimoveis.com.br/imovel/venda-apa...,900.0,"Rua Luiz Carlos Sarolli, 2021 - Recreio Dos Ba...",61.0,1200.0,2.0,1.0,2.0,1.0,510000.0
3,https://www.zapimoveis.com.br/imovel/venda-apa...,2700.0,"Rua Alberto de Faria - Leblon, Rio de Janeiro ...",307.0,1300.0,5.0,3.0,4.0,2.0,5800000.0
11,https://www.zapimoveis.com.br/imovel/venda-cas...,870.0,"Rua Iza Carrijo, 80 - Anil, Rio de Janeiro - RJ",225.0,2820.0,3.0,3.0,3.0,1.0,900000.0
14,https://www.zapimoveis.com.br/imovel/venda-apa...,670.0,"Rua Piauí, 77 - Todos os Santos, Rio de Janeir...",50.0,526.0,2.0,1.0,2.0,1.0,230000.0
19,https://www.zapimoveis.com.br/imovel/venda-apa...,821.0,"Rua Dois de Fevereiro - Engenho De Dentro, Rio...",76.0,650.0,3.0,1.0,3.0,1.0,235000.0


In [5]:
dataframe_imoveis_tratado.to_parquet('dataframe_imoveis_a_vender_tratados.parquet', index=False)